In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style='whitegrid')


# 1. Carregamento e ajuste de tipos

Carregamos a base, ajustamos tipos numéricos e categóricos, e padronizamos valores binários.


In [ ]:
df = pd.read_csv('CHURN_TELECON_MOD08_TAREFA.csv', delimiter=';')
num_cols = ['Tempo_como_Cliente', 'Pagamento_Mensal', 'Total_Pago']
for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
bin_cols = ['Casado', 'Dependents', 'PhoneService', 'Servico_Seguranca', 'Suporte_Tecnico', 'StreamingTV', 'Churn']
def normaliza_yn(x):
    if pd.isna(x): return np.nan
    x = str(x).strip().title()
    return 'Yes' if x in ['Yes', 'Sim'] else 'No'
for col in bin_cols:
    df[col] = df[col].apply(normaliza_yn).astype('category')
cat_cols = ['Genero', 'Servico_Internet', 'Tipo_Contrato', 'PaymentMethod']
for col in cat_cols:
    df[col] = df[col].str.strip().astype('category')
print(df.dtypes)


# 2A. Percentual de nulos

Calculamos o percentual de valores faltantes por coluna para decidir a melhor estratégia de tratamento.


In [ ]:
na_pct = (df.isna().mean()*100).round(2)
print('Percentual de nulos por coluna:')
print(na_pct)


# 2B. Exclusão de linhas sem Churn

Como Churn é a variável-alvo, linhas sem esse valor não podem ser usadas para modelagem ou avaliação.


In [ ]:
rows_before = len(df)
df = df[~df['Churn'].isna()].copy()
print(f'Removidas {rows_before-len(df)} linhas sem Churn')


# 2C. Imputação

Imputação dos valores faltantes:
- PhoneService: substituído por 'No' (ausência indica não contratação)
- Pagamento_Mensal: substituído pela mediana (distribuição assimétrica)
- Genero: substituído pela moda (valor mais frequente)


In [ ]:
df['PhoneService'].fillna('No', inplace=True)
df['Pagamento_Mensal'].fillna(df['Pagamento_Mensal'].median(), inplace=True)
df['Genero'].fillna(df['Genero'].mode()[0], inplace=True)
print('Nulos restantes após imputação:')
print(df.isna().sum())


# 3. Correção de valores inconsistentes

Padronização de grafias e caixa em colunas categóricas.


In [ ]:
map_genero = {'F':'Female', 'f':'Female', 'Feminino':'Female', 'M':'Male', 'm':'Male', 'Masculino':'Male'}
df['Genero'] = df['Genero'].replace(map_genero)
df['Servico_Internet'] = df['Servico_Internet'].replace({'dsl':'DSL', 'Dsl':'DSL', 'Fiber Optic':'Fiber optic'})
for col in bin_cols + cat_cols:
    df[col] = df[col].str.strip()
    df[col] = df[col].astype('category')
print('Valores únicos de Genero:', df['Genero'].unique())
print('Valores únicos de Servico_Internet:', df['Servico_Internet'].unique())


# 4. Padronização dos nomes das colunas

Padronização para snake_case.


In [ ]:
df.rename(columns=lambda c: c.strip().lower().replace(' ', '_'), inplace=True)
print('Colunas finais:', df.columns)


# 5. Gráficos

Visualização da distribuição de Pagamento Mensal e frequência de Gênero.


In [ ]:
plt.figure(figsize=(8,4))
sns.histplot(df['Pagamento_Mensal'], bins=30, kde=True, color='royalblue')
plt.axvline(df['Pagamento_Mensal'].mean(), color='red', linestyle='--', label='Média')
plt.axvline(df['Pagamento_Mensal'].median(), color='green', linestyle='-', label='Mediana')
plt.title('Distribuição de Pagamento Mensal')
plt.xlabel('Pagamento Mensal (R$)')
plt.ylabel('Contagem')
plt.legend()
plt.tight_layout()
plt.show()

plt.figure(figsize=(5,3))
sns.countplot(x='Genero', data=df, palette='pastel')
plt.title('Frequência de Gênero')
plt.xlabel('Gênero')
plt.ylabel('Contagem')
plt.tight_layout()
plt.show()


# 6. Resultado final

Base tratada, sem nulos, tipos corretos e categorias padronizadas.


In [ ]:
print('Shape final:', df.shape)
print('Total de nulos restantes:', df.isna().sum().sum())
df.head()
